<a href="https://colab.research.google.com/github/aliakbarbadri/persian-poetry-creator/blob/master/char-gru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import time

# Load the data

In [0]:
url = "https://raw.githubusercontent.com/aliakbarbadri/persian-poetry-creator/master/shahname2.txt"
filepath = keras.utils.get_file("shahname.txt", url) 
corpus = open(filepath, 'rb').read().decode(encoding='utf-8')

In [4]:
vocab = sorted(set(corpus))
print(len(vocab), "chars")

48 chars


In [0]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in corpus])

In [0]:
seq_length = 100
examples_per_epoch = len(corpus)//(seq_length+1)
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [8]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

# Model

In [0]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [0]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [10]:
model = keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[BATCH_SIZE, None]))
model.add(tf.keras.layers.GRU(rnn_units,return_sequences=True,stateful=True,
                              recurrent_initializer='glorot_uniform',
                              recurrent_regularizer=keras.regularizers.l2(0.01)
                              ))
model.add(tf.keras.layers.GRU(rnn_units,return_sequences=True,stateful=True,
                              recurrent_initializer='glorot_uniform',
                              recurrent_regularizer=keras.regularizers.l2(0.01)
                              ))
model.add(tf.keras.layers.Dense(vocab_size))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           12288     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
gru_1 (GRU)                  (64, None, 1024)          6297600   
_________________________________________________________________
dense (Dense)                (64, None, 48)            49200     
Total params: 10,297,392
Trainable params: 10,297,392
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [12]:
history = model.fit(dataset, epochs=10)

Epoch 1/10
395/395 [==============================] - 52s 131ms/step - loss: 3.2018 - accuracy: 0.3289
Epoch 2/10
395/395 [==============================] - 52s 131ms/step - loss: 1.9235 - accuracy: 0.4413
Epoch 3/10
395/395 [==============================] - 52s 131ms/step - loss: 1.7774 - accuracy: 0.4797
Epoch 4/10
395/395 [==============================] - 52s 131ms/step - loss: 1.6917 - accuracy: 0.5012
Epoch 5/10
395/395 [==============================] - 52s 131ms/step - loss: 1.6257 - accuracy: 0.5238
Epoch 6/10
395/395 [==============================] - 52s 131ms/step - loss: 1.5843 - accuracy: 0.5355
Epoch 7/10
395/395 [==============================] - 52s 131ms/step - loss: 1.5541 - accuracy: 0.5438
Epoch 8/10
395/395 [==============================] - 52s 131ms/step - loss: 1.5301 - accuracy: 0.5506
Epoch 9/10
395/395 [==============================] - 52s 131ms/step - loss: 1.5091 - accuracy: 0.5568
Epoch 10/10
395/395 [==============================] - 52s 131ms/step - l

In [13]:
main_model = keras.Sequential()
main_model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[1, None]))
main_model.add(tf.keras.layers.GRU(rnn_units,return_sequences=True,stateful=True,
                              recurrent_initializer='glorot_uniform',
                              recurrent_regularizer=keras.regularizers.l2(0.01)
                              ))
main_model.add(tf.keras.layers.GRU(rnn_units,return_sequences=True,stateful=True,
                              recurrent_initializer='glorot_uniform',
                              recurrent_regularizer=keras.regularizers.l2(0.01)
                              ))
main_model.add(tf.keras.layers.Dense(vocab_size))

main_model.set_weights(model.get_weights())
model = main_model
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            12288     
_________________________________________________________________
gru_2 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
gru_3 (GRU)                  (1, None, 1024)           6297600   
_________________________________________________________________
dense_1 (Dense)              (1, None, 48)             49200     
Total params: 10,297,392
Trainable params: 10,297,392
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text(model, start_string, temperature = 1.0):
  num_generate = 1000
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      predictions = tf.squeeze(predictions, 0)
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
      input_eval = tf.expand_dims([predicted_id], 0)
      text_generated.append(idx2char[predicted_id])
  return (start_string + ''.join(text_generated))

In [26]:
print(generate_text(model, start_string=u"که ایران چو باغی ست خرم بهار", temperature=1))

که ایران چو باغی ست خرم بهار
همانا نام من گونه یکی خواسته	هم خورد و برداشتی برخواند
هرنگ خوب اندر آید این تای پادشا
همه لشکر و دینار خواندند
ز شاها مباداب را بیابان من	برفتند ریزی از پیش برفرمان شدی
چنین تا که گیتی نمانش پیام
بیاید به نزدیک و تخم	نجویید که از خود به جنگ واژپوی
بپردازمگرد چنانچون بسود	ز هر سوی پیکرن کرد اوی
همه نیکان خواهرش پیش
درودش دمان	نگهبان مرا پهلواند
همه دشت آتش کنیم اوی	همی‌ریخت بر کس فرامرز داد
بباشم بوسید تا کرد جنگ	ندیدم بچنگ اندر دل خسته گندجاه
ز لشکر بسی گویم بدان کنیم	سپردار و نخجوی و مردم
سیاووش هوش نیسته مشکین و کمر
سپه را به زیر و پیش تویانی شورند
پر از راه بودند بروی آواز فرمان شاه	چو بشنید زین تیرگونه خورد
گره در خاکست و کو نامه را پیل	سپهدار تاک و افسر فریدون رسید
کنون انباره کوهه باز آمدند
به ایوانش برآمد روی زر	شما خواست تاج و تختی کنید
چنان دار گفت از بر سرشت	همی گر چندگاه بینی کنیم
ابا نامور گفتنی کار خواه	یکی پادشایستشان وران کجا آزرده‌ای
مباشید پن خوشیدن آهنین	بچیست مرگ آمدم بر سوی کوش
همان یکی نامش تنگش زودر شدست	که لشکر گنج و سپاهی ز کار دل
س